In [ ]:
import pandas as pd
import requests
from google.cloud import bigquery

In [ ]:
# Download CSV file
url = 'https://landregistry.data.gov.uk/app/ukhpi/download/new.csv?from=1975-01-01&to=2023-03-01&location=http%3A%2F%2Flandregistry.data.gov.uk%2Fid%2Fregion%2Funited-kingdom&thm%5B%5D=property_type&in%5B%5D=avg'

In [ ]:
#Load it into a DataFrame
df = pd.read_csv(url)
# Select desired columns and rename them
df = df[['Pivotable date', 'Average price All property types']]
df = df.rename(columns={'Pivotable date': 'Date', 'Average price All property types': 'AvgPrice'})
df

In [ ]:
# Save the edited DataFrame to a new CSV file
output_file = 'edited_average_prices.csv'
df.to_csv(output_file, index=False)

In [ ]:
# Load the CSV file to BigQuery table
project_id = 'project_id'
dataset_id = 'dataset_id'
table_name = 'table_name'

client = bigquery.Client(project=project_id)
dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_name)

job_config = bigquery.LoadJobConfig()
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
job_config.autodetect = True  # Infer table schema from CSV file

with open(output_file, 'rb') as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()  # Wait for the job to complete

print(f'Successfully loaded {job.output_rows} rows into {project_id}.{dataset_id}.{table_name}')